# Домашнее задание: Промптинг на Python

## Введение
В данном задании мы будем работать с API онлайн моделей через together.ai. Эти модели предоставляют $5 кредита при регистрации, что позволит вам провести необходимые эксперименты. Вначале мы познакомимся с API на практике, а затем выполним три основных задания.

---

## Задача 1: Знакомство с API together.ai (5 баллов)
1. Зарегистрируйтесь на платформе [together.ai](https://together.ai/) и получите API ключ.
2. Используйте приведенный ниже код для вызова модели Llama через together.ai:


In [1]:
import requests
import json

In [2]:
# settings
import os
from dotenv import load_dotenv

load_dotenv("../.env")

API_KEY = os.getenv("TOGEHTER_AI_API_KEY")
DEFAULT_MODEL_NAME: str = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
DEFAULT_MAX_TOKENS = 5000
LOG_RATE_LIMIT_RETRIES = False

In [3]:
# Вставьте свой API ключ
# API_KEY = "ваш_ключ_здесь"

# Параметры модели
url = "https://api.together.ai/v1/completions"
data = {
    "model": DEFAULT_MODEL_NAME,
    "prompt": "Translate the following English text to French: 'Hello, how are you?'",
    "max_tokens": 50
}
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

model_response = requests.post(url, headers=headers, json=data)
if model_response.status_code == 200:
    print("Response:", json.loads(model_response.text))
else:
    print("Error:", model_response.status_code, model_response.text)

Response: {'id': '90a00d3fe8d0f99c', 'object': 'text.completion', 'created': 1738225321, 'model': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', 'prompt': [], 'choices': [{'text': " is a common greeting in many languages. It is a polite way to ask about someone's well-being. The phrase is often used in informal settings, such as when meeting a friend or acquaintance. It is also used in formal settings, such as in", 'finish_reason': 'length', 'seed': 16735966801836636000, 'logprobs': None, 'index': 0}], 'usage': {'prompt_tokens': 15, 'completion_tokens': 50, 'total_tokens': 65}}


3. Модифицируйте запрос, чтобы:
   - Решить простую математическую задачу (например, сложение чисел).
   - Сгенерировать текст на тему "Как искусственный интеллект меняет мир".


In [4]:
# Common code
import time
import requests


def execute_llm_api_request(data: dict) -> dict:
    url = "https://api.together.ai/v1/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    defaults = {
        "model": DEFAULT_MODEL_NAME,
        "max_tokens": DEFAULT_MAX_TOKENS,
    }

    retry_count = 5
    sleep_sec = 1
    while True:
        response = requests.post(url, headers=headers, json={**defaults, **data})
        if response.status_code == 429: # rate limit hit - wait and retry
            if retry_count <= 0:
                response.raise_for_status()
            
            if LOG_RATE_LIMIT_RETRIES:
                print(f"Api warning: Rate limit exceeded, waiting {sleep_sec} second" + ("" if sleep_sec == 1 else "s"))
            
            time.sleep(sleep_sec)
            sleep_sec *= 2
            retry_count -= 1
        else:
            response.raise_for_status()    
            return json.loads(response.text)

In [5]:
# 1. Math

def solve_math_task_simple(math_task: str) -> str:
    chat_prompt = [
        {
            "role": "system",
            "content": """
Solve a mathematical task. Provide answer as a response with no intermediate steps.  Answer in English.

Example:
user: 15 * 3 + 5
assistant: 50
""".strip()
        },
        {
            "role": "user",
            "content": math_task
        },
    ]

    data = {
        "messages": chat_prompt
    }

    model_response = execute_llm_api_request(data)
    response = model_response["choices"][0]["text"]
    
    return response

# 1. Mathematical tasks
math_tasks = [
    "10 + 1",
    "5 * 15",
    "152 * 16",
    "123 + 234",
    "123 + 234 + 345",
    "123 + 234 + 345 + 456",
]

for expr in math_tasks:
    expected_answer = str(eval(expr))
    if (actual := solve_math_task_simple(expr)) == expected_answer:
        print(f"Correct answer: {actual}")
    else:
        print(f"Error. Expected {expected_answer} Actual {actual}")

# Выводы. Без использования chain of thought, модель нормально считает простые вычисления, но на больших числах или на большем количестве операций начинаются ошибки

Correct answer: 11
Correct answer: 75
Error. Expected 2432 Actual 2448
Correct answer: 357
Correct answer: 702
Error. Expected 1158 Actual 1128


In [6]:
# 2. Generate text

def generate_text(topic: str) -> str:
    chat_prompt = [
        {
            "role": "system",
            "content": "Generate text based on topic provided by user"
        },
        {
            "role": "user",
            "content": topic
        },
    ]

    data = {
        "messages": chat_prompt
    }

    model_response = execute_llm_api_request(data)
    response = model_response["choices"][0]["text"]
    
    return response

# Topic test
text = generate_text("How AI changes the world")
print(text)

**The Revolutionary Impact of AI on the World**

Artificial Intelligence (AI) has been transforming the world at an unprecedented pace, leaving an indelible mark on various aspects of our lives. From revolutionizing industries to transforming the way we interact with each other, AI has become an integral part of our daily lives. In this article, we'll explore the far-reaching impact of AI on the world and how it's changing the game.

**Industry Disruption**

AI has been disrupting traditional industries, creating new opportunities and challenges. Some of the key areas where AI is making a significant impact include:

1. **Healthcare**: AI-powered diagnostic tools are helping doctors detect diseases more accurately and at an early stage. AI-assisted robots are also performing surgeries with precision and speed.
2. **Finance**: AI-driven algorithms are analyzing vast amounts of data to predict market trends, detect fraud, and optimize investment strategies.
3. **Transportation**: Self-dr

## Задача 2: Решение математических задач через Chain of Thought (10 баллов)

Используя подход Chain of Thought (CoT), решите 10 математических задач и измерьте accuracy модели.


1. Создайте функцию, которая формирует запросы для модели с использованием CoT:

In [7]:
import re


def solve_math_task_cot(math_task: str) -> str:
    chat_prompt = [
        {
            "role": "system",
            "content": """
Solve a mathematical task, get to a solution step by step.

Follow the template:
Task: 10 + 12 * (16 + 1)
Observation:
1. Add 16 + 1 = 17
2. Multiply 12 * 17 = 204
3. Add 10 + 204 = 214
Final Answer: 214
""".strip()
        },
        {
            "role": "user",
            "content": math_task
        },
        # system message в конце помагает с большей вероятностью установить язык и формат ответа, но не всегда
        {
            "role": "system",
            "content": "Answer in English. Finish response with Final Answer: <single number>"
        },

    ]

    data = {
        "messages": chat_prompt
    }

    model_response = execute_llm_api_request(data)
    response_text = model_response["choices"][0]["text"]

    match = re.search(r"Final Answer:\s*(\d+)", response_text)

    if match:
        answer = match.group(1)
        return answer
    else:
        print(f"Unexpected model output for question {math_task}: {response_text}")
        return None

solve_math_task_cot("123 + 234 + 345 + 456"), solve_math_task_cot("Чему равно 23 умножить на 47?")

('1158', '1081')

2. Подготовьте 5 задач (например, из школьной программы) и выполните их решение через модель.

In [8]:
# Create math dataset

class MathDataset(dict):
    def add_eval(self, task: str) -> None:
        try:
            answer = eval(task)
        except Exception as e:
            raise ValueError(f"Cannot evaluate expected answer for {task}: {str(e)}")
        
        self[task] = answer


def create_math_dataset() -> MathDataset: 
    ds = MathDataset()
    ds.add_eval("5 * 15")
    ds.add_eval("152 * 16")
    ds.add_eval("123 + 234 + 345 + 456")
    ds["Чему равно 23 умножить на 47?"] = 1081
    ds["Sally is 54 years old and her mother is 80, how many years ago was Sally’s mother times her age?"] = 41
    ds["There is a three-digit number. The second digit is four times as big as the third digit, while the first digit is three less than the second digit. What is the number?"] = 141
    ds["Бабушка решила заняться фермерским хозяйством — выращивать и продавать помидоры. Она насобирала 100 кг томатов, погрузила их на тележку и выставила с утра перед домом. Помидоры, которые вырастила бабушка, на 99% состоят из воды, но на солнце часть воды испаряется сквозь кожуру. День выдался жарким, и к вечеру воды в помидорах стало уже 98%. Сколько теперь весят бабушкины помидоры?"] = 50
    ds["У Ильи дома есть часы со стрелками. Илья уходит на работу в 8 часов 00 минут утра. Домой Илья возвращается в 5 часов 30 минут вечера. Сколько раз за время отсутствия Ильи часовая и минутная стрелки успевают поравняться?"] = 50
    ds["Маша и Даша читают один и тот же роман. Маша за час прочитывает 20 страниц, а Даша – 21. Они одновременно начали читать роман, и Маша закончила читать позже Даши на 10 минут. Сколько страниц в романе?"] = 9

    return ds

3. Подсчитайте количество правильно решённых задач (accuracy).

In [9]:
# Evaluate
from tqdm.notebook import tqdm


def evaluate(answer_question_func: callable, dataset: dict) -> float:
    questions_total: int = 0
    questions_correct: int = 0

    for question, expected_answer in (iterable := tqdm(dataset.items())):
        answer = answer_question_func(question)

        questions_total += 1
        if answer != str(expected_answer):
            print(f"Wrong answer for question `{question}` is {answer}, expected {expected_answer}")
        else:
            questions_correct += 1

        iterable.set_description(f"{answer_question_func.__name__} accuracy: {(questions_correct / questions_total):.2f}")

    return questions_correct / questions_total

dataset = create_math_dataset()
accuracy_simple = evaluate(solve_math_task_simple, dataset)
accuracy_cot = evaluate(solve_math_task_cot, dataset)

print()
print("Evaluation results:")
print(f"Accuracy simple: {accuracy_simple:.2f}")
print(f"Accuracy COT: {accuracy_cot:.2f}")

  0%|          | 0/9 [00:00<?, ?it/s]

Wrong answer for question `152 * 16` is 2448, expected 2432
Wrong answer for question `123 + 234 + 345 + 456` is 1128, expected 1158
Wrong answer for question `Чему равно 23 умножить на 47?` is 1071, expected 1081
Wrong answer for question `Sally is 54 years old and her mother is 80, how many years ago was Sally’s mother times her age?` is 20 years ago., expected 41
Wrong answer for question `There is a three-digit number. The second digit is four times as big as the third digit, while the first digit is three less than the second digit. What is the number?` is 964, expected 141
Wrong answer for question `Бабушка решила заняться фермерским хозяйством — выращивать и продавать помидоры. Она насобирала 100 кг томатов, погрузила их на тележку и выставила с утра перед домом. Помидоры, которые вырастила бабушка, на 99% состоят из воды, но на солнце часть воды испаряется сквозь кожуру. День выдался жарким, и к вечеру воды в помидорах стало уже 98%. Сколько теперь весят бабушкины помидоры?` is

  0%|          | 0/9 [00:00<?, ?it/s]

Wrong answer for question `У Ильи дома есть часы со стрелками. Илья уходит на работу в 8 часов 00 минут утра. Домой Илья возвращается в 5 часов 30 минут вечера. Сколько раз за время отсутствия Ильи часовая и минутная стрелки успевают поравняться?` is 10, expected 50
Unexpected model output for question Маша и Даша читают один и тот же роман. Маша за час прочитывает 20 страниц, а Даша – 21. Они одновременно начали читать роман, и Маша закончила читать позже Даши на 10 минут. Сколько страниц в романе?: Let's break down the problem step by step:

1. Let the total number of pages in the book be x.
2. Since Маша за час прочитывает 20 страниц, за 10 минут она прочитает 20/60 = 1/3 страницы.
3. Поскольку Маша закончила читать позже Даши на 10 минут, то Даша прочитала на 10 минут больше, чем Маша. Итак, Даша прочитала 1/3 страницы за 10 минут, а Маша - за 0 минут (поскольку она закончила позже).
4. Поскольку Даша прочитала 1/3 страницы за 10 минут, то она прочитала 21 страниц за 30 минут (поск

## Задача 3: Классификация IMDB через few-shot и zero-shot (10 баллов)

Проведите классификацию отзывов IMDB на позитивные и негативные с использованием few-shot и zero-shot подходов.


1. Выберите 5 примеров для few-shot обучения (например, 2 позитивных и 3 негативных отзыва).
2. Реализуйте запросы к модели в режиме zero-shot и few-shot:

In [10]:
def build_classify_review_prompt(review: str, examples: dict = None) -> list[dict]:
    examples_prompt = ""
    if examples:
        examples_prompt = "Examples:\n"
        for review_text, sentiment in examples.items():
            examples_prompt += f"Review: `{review_text}` is {sentiment}\n"

    chat_prompt = [
        {
            "role": "system",
            "content": f"""
Classify movie review into positive or negative.
Respond with single word only - positive or negative.

{examples_prompt}
            """.strip()
        },
        {
            "role": "user",
            "content": "Review text: " + review
        },
    ]

    return chat_prompt


def classify_review(prompt: str, examples: dict=None) -> str:
    few_shot_prompt = build_classify_review_prompt(prompt, examples)

    data = {
        "messages": few_shot_prompt
    }
    
    model_response = execute_llm_api_request(data)
    response_text = model_response["choices"][0]["text"]
    
    return response_text.lower().strip()

In [11]:
from datasets import load_dataset

imdb = load_dataset("stanfordnlp/imdb")
seed = 42

In [12]:
def create_examples() -> dict:
    positive_examples = imdb["train"].shuffle(seed=seed).filter(lambda entry: entry["label"] == 1).select(range(2))
    negative_examples = imdb["train"].shuffle(seed=seed).filter(lambda entry: entry["label"] == 0).select(range(3))

    examples = {}
    for text, label in zip(positive_examples["text"] + negative_examples["text"], positive_examples["label"] + negative_examples["label"]):
        examples[text] = "positive" if label == 1 else "negative"

    return examples

def create_eval_data(num_items: int) -> dict:
    items = imdb["train"].shuffle(seed=seed).select(range(num_items))
    result = {item["text"]: "positive" if item["label"] == 1 else "negative" for item in items}

    return result

In [14]:
from functools import partial

num_evaluate_items = 100

multi_shot_classifier = partial(classify_review, examples = create_examples()); multi_shot_classifier.__name__ = "multi_shot_classifier"
zero_shot_classifier = partial(classify_review, examples = None); zero_shot_classifier.__name__ = "zero_shot_classifier"

eval_data = create_eval_data(num_evaluate_items)

accuracy_multi_shot = evaluate(multi_shot_classifier, eval_data)
accuracy_single_shot = evaluate(multi_shot_classifier, eval_data)

print()
print("Results:")
print(f"Single shot accuracy: {accuracy_single_shot}")
print(f"Multi shot accuracy: {accuracy_multi_shot}")

  0%|          | 0/100 [00:00<?, ?it/s]

Wrong answer for question `In the process of trying to establish the audiences' empathy with Jake Roedel (Tobey Maguire) the filmmakers slander the North and the Jayhawkers. Missouri never withdrew from the Union and the Union Army was not an invading force. The Southerners fought for State's Rights: the right to own slaves, elect crooked legislatures and judges, and employ a political spoils system. There's nothing noble in that. The Missourians could have easily traveled east and joined the Confederate Army.<br /><br />It seems to me that the story has nothing to do with ambiguity. When Jake leaves the Bushwhackers, it's not because he saw error in his way, he certainly doesn't give himself over to the virtue of the cause of abolition.` is negative, expected positive
Wrong answer for question `I love horses and admire hand drawn animation, so I expected nothing short of amazement from Dreamworks new animated picture Spirit: Stallion of the Cimarron. I guess you could say I was a litt

  0%|          | 0/100 [00:00<?, ?it/s]

Wrong answer for question `In the process of trying to establish the audiences' empathy with Jake Roedel (Tobey Maguire) the filmmakers slander the North and the Jayhawkers. Missouri never withdrew from the Union and the Union Army was not an invading force. The Southerners fought for State's Rights: the right to own slaves, elect crooked legislatures and judges, and employ a political spoils system. There's nothing noble in that. The Missourians could have easily traveled east and joined the Confederate Army.<br /><br />It seems to me that the story has nothing to do with ambiguity. When Jake leaves the Bushwhackers, it's not because he saw error in his way, he certainly doesn't give himself over to the virtue of the cause of abolition.` is negative, expected positive
Wrong answer for question `I love horses and admire hand drawn animation, so I expected nothing short of amazement from Dreamworks new animated picture Spirit: Stallion of the Cimarron. I guess you could say I was a litt

In [ ]:
# Пример zero-shot классификации
review_prompt = "Этот фильм был потрясающим! Сюжет увлекательный, а актеры великолепны."
zero_shot_prompt = f"Классифицируйте следующий отзыв как позитивный или негативный: {review_prompt}"

data = {
    "prompt": zero_shot_prompt,
    "max_tokens": 50
}

result = execute_llm_api_request(data)
print(result["choices"][0]["text"].strip())

Я не мог смотреть в глаза своим друзьям, потому что я так сильно плакал. Это был действительно потрясающий фильм.**

Предположим, что вы хотите написать ответ на этот отзыв


3. Сравните результаты, объяснив различия между zero-shot и few-shot подходами.

## Задача 4: Self-reflection и качество ответов модели (10 баллов)

Проверьте, как self-reflection влияет на качество ответов модели.


1. Реализуйте функцию self-reflection, которая анализирует ответ модели и предлагает улучшения:

In [13]:
def self_reflection(prompt: str) -> str:
    reflection_prompt = f"Проанализируйте ответ и предложите улучшения: {prompt}"
    # Подставьте сюда вызов API
    return response_text

2. Используйте self-reflection для 5 задач из задачи 2 (CoT) и сравните результаты до и после рефлексии.
3. Ответьте на вопросы:
   - Улучшаются ли ответы?
   - Исправляет ли модель правильные ответы на неправильные?

## Задача 5: Защита от инъекций (10 баллов)

 Исследуйте методы защиты от инъекций в пользовательских вводах.


1. Реализуйте функцию, которая проверяет ввод пользователя на наличие потенциальных инъекций:

In [14]:
import re

# Функция проверки на инъекцию
def detect_injection(user_input: str) -> bool:
    """
    Проверяет текст на наличие возможных инъекций.
    Возвращает True, если найдена инъекция.
    """
    # Примеры подозрительных шаблонов
    # Подобные шаблоны бессмысленны потому что
    # 1. bad recall - мало какой хак будет основан 
    injection_patterns = [
        r"ignore.*instructions",  # Игнорировать инструкции
        r"forget.*previous",      # Забыть предыдущие команды
        r"reveal.*secret",        # Раскрыть секрет
        r"break.*rules",          # Нарушить правила
    ]
    for pattern in injection_patterns:
        if re.search(pattern, user_input, re.IGNORECASE):
            return ...
    return False

# Пример использования
def process_user_input(user_input: str) -> str:
    """
    Обрабатывает пользовательский ввод с проверкой на инъекции.
    """
    if detect_injection(user_input):
        return "Ошибка: обнаружена потенциальная инъекция!"

    # Если инъекций нет, отправляем запрос к модели
    data = {
        "model": "NousResearch/Meta-Llama-3.1-8B",
        "prompt": user_input,
        "max_tokens": 50
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return json.loads(response.text)["choices"][0]["text"].strip()
    else:
        return f"Ошибка: {response.status_code}"


Подобные шаблоны бессмысленны потому что

1. bad recall - мало какой хак будет основан на таких простых словах как выше указано, плюс есть синонимы и фразы означающие тот же смысл, я молчу про разные языки, попытки завернуть вопрос в base64, написать текст с каждой буквой в слове через символ
2. bad precision - регекс сделан так, что любое количество символов между может быть между словами, то есть "do not break any rules" тоже будет распознано как Jailbreak



2. Протестируйте функцию на 5 различных вводах, включая как корректные запросы, так и попытки инъекций.
3. Напишите выводы о том, как система справляется с защитой и какие улучшения можно внести.

In [18]:
# Тестирование
inputs = [
    "What is the capital of France?",
    ...
]

for i, inp in enumerate(inputs):
    print(f"Input {i+1}: {inp}")
    print(f"Output: {process_user_input(inp)}")
    print("-" * 40)

Input 1: What is the capital of France?
Output: Ошибка: 404
----------------------------------------
Input 2: Ellipsis


TypeError: expected string or bytes-like object, got 'ellipsis'

: 

## Требования к оформлению
- Каждый результат должен быть сопровожден кодом, комментариями и выводами.
- Предоставьте accuracy, сравнения и выводы в формате markdown в jupyter notebook.

## Дополнительное задание (по желанию, +5 баллов)
Проверьте, как работает модель с разными длинами промпта (от коротких до детализированных). Как длина промпта влияет на качество ответа?

---

**Удачи в выполнении задания!**
